# Prepare[需要运行]

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

#Load Drive[需要运行]

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# data prepare

In [24]:
corpus_name = "qa_corpus.json"
corpus = os.path.join("/content/drive/MyDrive", corpus_name)
import json
def printLines(file, n=10):
    with open(file, encoding='utf-8') as datafile:
        lines = datafile.readlines()
        # lines=json.load(datafile)
    allline = []
    for line in lines[:n]:
      
      #print(line)
      a=json.loads(line)
      # print(a['question'],a['answers'][0])
      # print(a)
      delimiter = '\t'
      
      
      for ans in a['answers']:
        lineObj = []
        lineObj=a['question']+delimiter+ans
        # print(lineObj)
        allline.append(lineObj)
        
    print(allline)  

printLines(corpus)

['交通事故中，保险公司及肇事者司机方全部不闻不问。把伤者拖在医院，一直由家属自行在治疗请问律师该如何处理？\t可治疗结束后，起诉解决。', '我4月骑摩托车撞人已付医药费用二万多，现在她进行十级伤残鉴定，要求赔偿金5万怎么办？\t你可以去应诉', '你好，请问如果在外地治疗病而不是医保定点医院，拿回老家能报医保吗\t农医保看门诊，每年可以报一部分，凭发票，处方或检查报告，拿到自己当地可以报销', '我的情况特殊，我家五楼买了六年了，未装修，未入住，上下水也没有开请问责任在谁？\t主管道属于物业管理范围，物业有责任。', '我姐姐被儿子媳妇打了三次我姐不识字叫我报警我就打了\t你好，轻伤涉嫌犯罪的', '古钱币登记需要到什么部门去登记，登记需不需要什么费用\t如果属于文物范畴，肯定需要的！因为国家文物法明确规定：中国人不准私自将文物卖给外国人！最好不要触犯法律。', '法院拍卖的房子那种情况不能过户？\t过户流程为，现场竞买成功后，买受人应立即支付成交价一定比例的款项作为定金，并在拍卖行协助下与委托人签订拍卖房地产转让合同书。买受人再支付全部价款后，凭转让合同书和相关证明文件到房地产登记机关办理产权过户手续，取得房地产权证。房屋司法拍卖流程一、接受拍卖委托在这一阶段，委托人将有意要拍卖的房产明确委托给拍卖行，双方签订委托房产拍卖协议，对委托拍卖的标的物达成基本意向。委托人在委托时一般要想拍卖行提供房地产权证、身份证等相关房产证明文件。二、拍卖房产标的的调查确认拍卖行对房产委托人提供的房地产产权证明、有关文件、房产证明材料等进一步核实，必要时到相关部门调查取证，同时还必须对房产进行现场勘查。三、接受委托、签订委托、拍卖合同书经调查确认后，拍卖行认为符合拍卖委托条件的，与委托人签订委托拍卖合同。委托拍卖必须符合《拍卖法》的要求。委托拍卖合同中要对拍卖房地产的情况、拍卖费用、拍卖方式和期限、违约责任等加以明确。四、房地产拍卖底价的确定拍卖行对房地产市场进行调查与分析，必要时请专业的房地产估价人员对拍卖房地产进行价格评估，与委托方共同商谈，后确定拍卖底价和起拍价。五、发布拍卖公告，组织接待竞买人拍卖行一般要在拍卖日前半个月至一个月前登报或通过电视媒体以公告形式发布拍卖房地产的信息，拍卖行要对公告的内容真实性负责。同时，组织接待竞买人，向竞买人提供资料，审查竞买人资格，收取保

In [28]:
def loadLines(fileName):
    lines = []
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
          values=json.loads(line)
          for ans in values['answers']:
            lineObj = []
            lineObj=[values['question'],ans]
            # print(lineObj)
            lines.append(lineObj)
    print(lines)
    return lines

In [29]:
datafile = os.path.join("/content/drive/MyDrive", "lawqa1.txt")
delimiter = '\t'
# 对分隔符delimiter进行decode，这里对tab进行decode结果并没有变
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

In [31]:
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')

    for pair in loadLines(corpus):
        print(pair)
        writer.writerow(pair)



Writing newly formatted file...


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



流式输出内容被截断，只能显示最后 5000 行内容。
['我与合作方签署合同领导同意了 最后出现漏洞 对方公司不履行或者加钱 公', '看具体情况，代表公司，应当由公司承担']
['我与合作方签署合同领导同意了 最后出现漏洞 对方公司不履行或者加钱 公', '不合理']
['我与合作方签署合同领导同意了 最后出现漏洞 对方公司不履行或者加钱 公', '不合理 经营风险 后果应该由公司承担']
['我与合作方签署合同领导同意了 最后出现漏洞 对方公司不履行或者加钱 公', '不应当由员工承担']
['我和我妻子生活了十二年。期间我父母出资盖了一趟房子。房主是我父亲名字。算我们夫妻的共同财产吗？', '你好，夫妻双方在婚姻关系存续期间所得的财产，为夫妻共同财产，包括：(1)一方或双方劳动所得的收入和购置的财产;(2)一方或双方继承、受赠的财产；(3)一方或双方由知识产权取得的经济利益;(4)一方或双方从事承包、租赁等生产、经营活动的收益；(5)一方或双方取得的债权;(6)一方或双方的其他合法所得。']
['我和我妻子生活了十二年。期间我父母出资盖了一趟房子。房主是我父亲名字。算我们夫妻的共同财产吗？', '无关。']
['我和我妻子生活了十二年。期间我父母出资盖了一趟房子。房主是我父亲名字。算我们夫妻的共同财产吗？', '你好，《中华人民共和国婚姻法》及相关法律规定，夫妻共同财产具体主要指：夫妻关系存续期间一方或双方的各项合法收入以及由该收入转化而成的各项财产和财产性权利。夫妻关系存续期间自结婚登记之日起至离婚登记或离婚判决生效。具体的应包括：(一)夫妻双方各自的工资、奖金;(二)生产、经营性的收益;(三)知识产权的收益：是指婚姻关系存续期间，实际取得或者已经明确可以取得的财产性收益;(四)继承或接受赠予所得的财产;(五)军人名下的复员费、自主择业费等一次性费用的，以夫妻婚姻关系存续年限乘以年平均值的所得额;(六)一方以个人财产投资取得的收益;(七)男女双方实际取得或者应当取得的住房补贴、住房公积金;(八)男女双方实际取得或者应当取得的养老保险金、破产安置补偿费']
['以谈恋爱名义，认识半个月骗钱骗色，被骗走20000块，之后消失，报警说是经济纠纷怎么办？', '你好，你走民事诉讼']
['以谈恋爱名义，认识半个月骗钱骗色，被骗走20000块，之后消失，报警说是经济纠

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



流式输出内容被截断，只能显示最后 5000 行内容。
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '报警处理']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '委托律师起诉处理']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '到法院起诉离婚。']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '您可以委托律师，向法院起诉离婚。想帮忙的，联系。']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '起诉离婚，家暴是法定离婚情节']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '及时报警']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '可以报警。同时可以起诉离婚。']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '可起诉离婚']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '受到家庭暴力，及时采取如下措施：1、建议及时报警，保护自己人身安全并固定证据。2、伤情较重要求验伤并到医院治疗。这样都可作为离婚证据。3、协商不成，你可起诉离婚并要求对方赔偿。若需帮助，可来电或预约当面洽谈。']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '起诉离婚吧']
['我想和我老公离婚，他不离，因为他还有一个弟弟，他爸妈不好好上班赚钱，还要我们的钱有什么好办法？', '回电详谈']
['我想请问下户主死后  妻子能买卖房子吗  需要得到三个子女同意吗', '要看房产证上是谁的名字']
['我想请问下户主死后  妻子能买卖房子吗  需要得到三个子女同意吗', '请问房子产证上是谁的名字']
['我想请问下户主死后  妻子能买卖房子吗  需要得到三个子女同意吗', '你好，如果子女可以继承该房产，卖房子需要

In [ ]:
loadLines(corpus)

# voc[需要运行]

In [32]:
import jieba


In [33]:
# 预定义的token
PAD_token = 0  # 表示padding 
SOS_token = 1  # 句子的开始 
EOS_token = 2  # 句子的结束 

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # 目前有SOS, EOS, PAD这3个token。

    def addSentence(self, sentence):
        for word in jieba.cut(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # 删除频次小于min_count的token 
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # 重新构造词典 
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens
        
        # 重新构造后词频就没有意义了(都是1)
        for word in keep_words:
            self.addWord(word)

    def delnum(self):
      keep_words = []

      for k, v in self.word2count.items():
            if k.isdigit is False:
                keep_words.append(k)
      
      

In [34]:
MAX_LENGTH = 100  # 句子最大长度是10个词(包括EOS等特殊词)

# 把Unicode字符串变成ASCII
# 参考https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )
 
def normalizeString(s):
    # 变成小写、去掉前后空格，然后unicode变成ascii
    # s = unicodeToAscii(s.lower().strip())
    # 在标点前增加空格，这样把标点当成一个词
    # s = re.sub(r"([.!?])", r" \1", s)
    # 字母和标点之外的字符都变成空格
    # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    # 因为把不用的字符都变成空格，所以可能存在多个连续空格
    # 下面的正则替换把多个空格变成一个空格，最后去掉前后空格
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# 读取问答句对并且返回Voc词典对象 
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # 文件每行读取到list lines中。 
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # 每行用tab切分成问答两个句子，然后调用normalizeString函数进行处理。
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p):
    return (len(p)==2) and (len(jieba.lcut(p[0])) < MAX_LENGTH and len(jieba.lcut(p[1])) < MAX_LENGTH)

#过滤非法的
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# 使用上面的函数进行处理，返回Voc对象和句对的list 
def loadPrepareData(corpus, corpus_name, datafile):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        # print(pair)
        
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
        
        
    print("Counted words:", voc.num_words)
    return voc, pairs




# make voc

In [35]:
# Load/Assemble voc and pairs
voc, pairs = loadPrepareData(corpus, corpus_name, datafile)
# 输出一些句对
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 599031 sentence pairs
Trimmed to 572414 sentence pairs
Counting words...
Counted words: 97078

pairs:
['交通事故中，保险公司及肇事者司机方全部不闻不问。把伤者拖在医院，一直由家属自行在治疗请问律师该如何处理？', '可治疗结束后，起诉解决。']
['我4月骑摩托车撞人已付医药费用二万多，现在她进行十级伤残鉴定，要求赔偿金5万怎么办？', '你可以去应诉']
['你好，请问如果在外地治疗病而不是医保定点医院，拿回老家能报医保吗', '农医保看门诊，每年可以报一部分，凭发票，处方或检查报告，拿到自己当地可以报销']
['我的情况特殊，我家五楼买了六年了，未装修，未入住，上下水也没有开请问责任在谁？', '主管道属于物业管理范围，物业有责任。']
['我姐姐被儿子媳妇打了三次我姐不识字叫我报警我就打了', '你好，轻伤涉嫌犯罪的']
['古钱币登记需要到什么部门去登记，登记需不需要什么费用', '如果属于文物范畴，肯定需要的！因为国家文物法明确规定：中国人不准私自将文物卖给外国人！最好不要触犯法律。']
['网贷忘还了过了好多年现在征信出现呆帐了然后我们把钱还上了可是征信仍然处于呆账状态怎么能消除啊', '网上一些网贷平台宣称黑户也能办，这里不建议随便申请，以防被骗高额手续费。']
['网贷忘还了过了好多年现在征信出现呆帐了然后我们把钱还上了可是征信仍然处于呆账状态怎么能消除啊', '你好，信用卡呆账即是征信黑名单了，即使已经还清信用卡。一般情况下也是无法办理贷款业务的。大部分贷款公司以及网贷平台都需要查征信的。']
['我是被告家属出了交通事故请律师得多少钱。', '需要根据情况，可以联系我']
['我前两天订了一辆车，但是车还没定，钱已经交了，只交了定金，定金可不可以退退？', '单方违约，定金可以不退还。']


In [36]:
MIN_COUNT = 3    # 阈值为3


def trimRareWords(voc, pairs, MIN_COUNT):
    # voc.delnum()
    # 去掉voc中频次小于3的词 
    voc.trim(MIN_COUNT)
    
    # 保留的句对 
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # 检查问题
        for word in jieba.cut(input_sentence):
            if word not in voc.word2index:
                keep_input = False
                break
        # 检查答案
        for word in jieba.cut(output_sentence):
            if word not in voc.word2index:
                keep_output = False
                break

        # 如果问题和答案都只包含高频词，我们才保留这个句对
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), 
		len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# 实际进行处理
pairs = trimRareWords(voc, pairs, MIN_COUNT)



keep_words 55830 / 97075 = 0.5751
Trimmed from 572414 pairs to 531384, 0.9283 of total


# save voc & pair

In [37]:
clean_data = {
    'pairs':pairs ,
    'voc':voc,
}
save_clearpath = '/content/drive/MyDrive/corpus100.pth'
torch.save(clean_data, save_clearpath)
print('save clean data in %s' % save_clearpath)

save clean data in /content/drive/MyDrive/corpus100.pth


In [38]:
voc.word2index

{'交通事故': 3,
 '中': 4,
 '，': 5,
 '保险公司': 6,
 '及': 7,
 '肇事者': 8,
 '司机': 9,
 '方': 10,
 '全部': 11,
 '不闻不问': 12,
 '。': 13,
 '把': 14,
 '伤者': 15,
 '拖': 16,
 '在': 17,
 '医院': 18,
 '一直': 19,
 '由': 20,
 '家属': 21,
 '自行': 22,
 '治疗': 23,
 '请问': 24,
 '律师': 25,
 '该': 26,
 '如何': 27,
 '处理': 28,
 '？': 29,
 '可': 30,
 '结束': 31,
 '后': 32,
 '起诉': 33,
 '解决': 34,
 '我': 35,
 '4': 36,
 '月': 37,
 '骑': 38,
 '摩托车': 39,
 '撞': 40,
 '人': 41,
 '已付': 42,
 '医药': 43,
 '费用': 44,
 '二万多': 45,
 '现在': 46,
 '她': 47,
 '进行': 48,
 '十级': 49,
 '伤残': 50,
 '鉴定': 51,
 '要求': 52,
 '赔偿金': 53,
 '5': 54,
 '万': 55,
 '怎么办': 56,
 '你': 57,
 '可以': 58,
 '去': 59,
 '应诉': 60,
 '你好': 61,
 '如果': 62,
 '外地': 63,
 '不是': 64,
 '医保': 65,
 '定点医院': 66,
 '拿': 67,
 '回老家': 68,
 '能报': 69,
 '吗': 70,
 '农': 71,
 '看': 72,
 '门诊': 73,
 '每年': 74,
 '报': 75,
 '一部分': 76,
 '凭': 77,
 '发票': 78,
 '处方': 79,
 '或': 80,
 '检查报告': 81,
 '拿到': 82,
 '自己': 83,
 '当地': 84,
 '报销': 85,
 '的': 86,
 '情况': 87,
 '特殊': 88,
 '我家': 89,
 '五楼': 90,
 '买': 91,
 '了': 92,
 '六年': 93,
 '未': 94,
 '装修': 95,
 '

In [ ]:
pairs[10]

#load voc&pair[需要运行]

In [ ]:
save_clearpath = '/content/drive/MyDrive/corpus10.pth'
clean_data=torch.load(save_clearpath)


voc=clean_data['voc']
pairs=clean_data['pairs']

# Start

In [39]:
# 把句子的词变成ID
def indexesFromSentence(voc, sentence):
    try:
      # print("pass"+sentence)
      
      return [voc.word2index[word] for word in jieba.cut(sentence)] + [EOS_token]
    except:
      print("出错"+sentence)
      return [voc.word2index[word] for word in jieba.cut(sentence)] + [EOS_token]


# l是多个长度不同句子(list)，使用zip_longest padding成定长，长度为最长句子的长度。
def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

# l是二维的padding后的list
# 返回m和l的大小一样，如果某个位置是padding，那么值为0，否则为1
def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# 把输入句子变成ID，然后再padding，同时返回lengths这个list，标识实际长度。
# 返回的padVar是一个LongTensor，shape是(batch, max_length)，
# lengths是一个list，长度为(batch,)，表示每个句子的实际长度。
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])

    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# 对输出句子进行padding，然后用binaryMatrix得到每个位置是padding(0)还是非padding，
# 同时返回最大最长句子的长度(也就是padding后的长度)
# 返回值padVar是LongTensor，shape是(batch, max_target_length)
# mask是ByteTensor，shape也是(batch, max_target_length)
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# 处理一个batch的pair句对 
def batch2TrainData(voc, pair_batch):
    # 按照句子的长度(词数)排序
    pair_batch.sort(key=lambda x: len((x[0])), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        # print(pair[0])
        # print(pair[1])
        # print("---")
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths.cpu(), output, mask, max_target_len


# 示例
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 2435,    35,  1897,  6822,    61],
        [10391,   187, 19686,  5084,     5],
        [ 1202,   292,   286,  5085,    35],
        [ 3442,  4428,    91,  2512,   347],
        [    5,    86,    86,     5,    24],
        [  490,  2287,  5447,    62,   902],
        [ 2973,     5,     5,  8183,    13],
        [  513,   345,  2608,  2225,  1769],
        [33111,    35,   920,  2068,   324],
        [ 2161,   152,   920,     5,     5],
        [    5,  1851,   120, 10514,    35],
        [  350,     5,   370, 34442,    19],
        [ 1768,   571,    92,    86, 12247],
        [    5,  2304,   160,  7387, 31865],
        [ 2128,     5,     5,     5,   350],
        [   11,   116,   586,   665,     5],
        [  201,  4820,    59,  6317,  2535],
        [  353,     5,  1737,   239,    58],
        [    5,  6191, 18686,    55,  2173],
        [   35,  2693,   979,    36,   900],
        [  125,   810,     5,  2535,    92],
        [  204,    35,  1736,    97,   

In [40]:

class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # 初始化GRU，这里输入和hidden大小都是hidden_size，这里假设embedding层的输出大小是hidden_size
        # 如果只有一层，那么不进行Dropout，否则使用传入的参数dropout进行GRU的Dropout。
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # 输入是(max_length, batch)，Embedding之后变成(max_length, batch, hidden_size)
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        # 因为RNN(GRU)要知道实际长度，所以PyTorch提供了函数pack_padded_sequence把输入向量和长度
        # pack到一个对象PackedSequence里，这样便于使用。
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu(),enforce_sorted=False)
        # 通过GRU进行forward计算，需要传入输入和隐变量
        # 如果传入的输入是一个Tensor (max_length, batch, hidden_size)
        # 那么输出outputs是(max_length, batch, hidden_size*num_directions)。
        # 第三维是hidden_size和num_directions的混合，它们实际排列顺序是num_directions在前面，
        # 因此我们可以使用outputs.view(seq_len, batch, num_directions, hidden_size)得到4维的向量。
        # 其中第三维是方向，第四位是隐状态。
        
        # 而如果输入是PackedSequence对象，那么输出outputs也是一个PackedSequence对象，我们需要用
        # 函数pad_packed_sequence把它变成shape为(max_length, batch, hidden*num_directions)的向量以及
        # 一个list，表示输出的长度，当然这个list和输入的input_lengths完全一样，因此通常我们不需要它。
        outputs, hidden = self.gru(packed, hidden)
        # 参考前面的注释，我们得到outputs为(max_length, batch, hidden*num_directions)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # 我们需要把输出的num_directions双向的向量加起来
        # 因为outputs的第三维是先放前向的hidden_size个结果，然后再放后向的hidden_size个结果
        # 所以outputs[:, :, :self.hidden_size]得到前向的结果
        # outputs[:, :, self.hidden_size:]是后向的结果
        # 注意，如果bidirectional是False，则outputs第三维的大小就是hidden_size，
        # 这时outputs[:, : ,self.hidden_size:]是不存在的，因此也不会加上去。
        # 对Python slicing不熟的读者可以看看下面的例子：
        
        # >>> a=[1,2,3]
        # >>> a[:3]
        # [1, 2, 3]
        # >>> a[3:]
        # []
        # >>> a[:3]+a[3:]
        # [1, 2, 3]
        
        # 这样就不用写下面的代码了：
        # if bidirectional:
        #     outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # 返回最终的输出和最后时刻的隐状态。 
        return outputs, hidden

In [41]:
# Luong 注意力layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        # 输入hidden的shape是(1, batch=64, hidden_size=500)
        # encoder_outputs的shape是(input_lengths=10, batch=64, hidden_size=500)
        # hidden * encoder_output得到的shape是(10, 64, 500)，然后对第3维求和就可以计算出score。
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), 
				      encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)
    
    # 输入是上一个时刻的隐状态hidden和所有时刻的Encoder的输出encoder_outputs
    # 输出是注意力的概率，也就是长度为input_lengths的向量，它的和加起来是1。
    def forward(self, hidden, encoder_outputs):
        # 计算注意力的score，输入hidden的shape是(1, batch=64, hidden_size=500)，
        # 表示t时刻batch数据的隐状态
        # encoder_outputs的shape是(input_lengths=10, batch=64, hidden_size=500) 
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            # 计算内积，参考dot_score函数
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        # 把attn_energies从(max_length=10, batch=64)转置成(64, 10)
        attn_energies = attn_energies.t()

        # 使用softmax函数把score变成概率，shape仍然是(64, 10)，然后用unsqueeze(1)变成
        # (64, 1, 10) 
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [42]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 保存到self里，attn_model就是前面定义的Attn类的对象。
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 定义Decoder的layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 注意：decoder每一步只能处理一个时刻的数据，因为t时刻计算完了才能计算t+1时刻。
        # input_step的shape是(1, 64)，64是batch，1是当前输入的词ID(来自上一个时刻的输出)
        # 通过embedding层变成(1, 64, 500)，然后进行dropout，shape不变。
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 把embedded传入GRU进行forward计算
        # 得到rnn_output的shape是(1, 64, 500)
        # hidden是(2, 64, 500)，因为是两层的GRU，所以第一维是2。
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 计算注意力权重， 根据前面的分析，attn_weights的shape是(64, 1, 10)
        attn_weights = self.attn(rnn_output, encoder_outputs)
        
        # encoder_outputs是(10, 64, 500) 
        # encoder_outputs.transpose(0, 1)后的shape是(64, 10, 500)
        # attn_weights.bmm后是(64, 1, 500)
        
        # bmm是批量的矩阵乘法，第一维是batch，我们可以把attn_weights看成64个(1,10)的矩阵
        # 把encoder_outputs.transpose(0, 1)看成64个(10, 500)的矩阵
        # 那么bmm就是64个(1, 10)矩阵 x (10, 500)矩阵，最终得到(64, 1, 500)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # 把context向量和GRU的输出拼接起来
        # rnn_output从(1, 64, 500)变成(64, 500)
        rnn_output = rnn_output.squeeze(0)
        # context从(64, 1, 500)变成(64, 500)
        context = context.squeeze(1)
        # 拼接得到(64, 1000)
        concat_input = torch.cat((rnn_output, context), 1)
        # self.concat是一个矩阵(1000, 500)，
        # self.concat(concat_input)的输出是(64, 500)
        # 然后用tanh把输出返回变成(-1,1)，concat_output的shape是(64, 500)
        concat_output = torch.tanh(self.concat(concat_input))

        # out是(500, 词典大小=7826)    
        output = self.out(concat_output)
        # 用softmax变成概率，表示当前时刻输出每个词的概率。
        output = F.softmax(output, dim=1)
        # 返回 output和新的隐状态 
        return output, hidden

In [43]:
def maskNLLLoss(inp, target, mask):
    # 计算实际的词的个数，因为padding是0，非padding是1，因此sum就可以得到词的个数
    nTotal = mask.sum()

    
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [44]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # 梯度清空
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # 设置device，从而支持GPU，当然如果没有GPU也能工作。
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # print("input_variable",input_variable)
    # print("target_variable",target_variable)

    # 初始化变量
    loss = 0
    print_losses = []
    n_totals = 0

    # encoder的Forward计算
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Decoder的初始输入是SOS，我们需要构造(1, batch)的输入，表示第一个时刻batch个输入。
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # 注意：Encoder是双向的，而Decoder是单向的，因此从下往上取n_layers个
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # 确定是否teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # 一次处理一个时刻 
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: 下一个时刻的输入是当前正确答案
            decoder_input = target_variable[t].view(1, -1)
            # print("decoder_input",decoder_input)
            # 计算累计的loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            # print("mask_loss",mask_loss)
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # 不是teacher forcing: 下一个时刻的输入是当前模型预测概率最高的值
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # 计算累计的loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # 反向计算 
    loss.backward()

    # 对encoder和decoder进行梯度裁剪
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # 更新参数
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [45]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, 
              embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, 
              print_every, save_every, clip, corpus_name, loadFilename):

    # 随机选择n_iteration个batch的数据(pair)
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # 初始化
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # 训练
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # 训练一个batch的数据
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        # print("loss:",loss)
        print_loss += loss

        # 进度
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}"
			.format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # 保存checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'
		.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [46]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Encoder的Forward计算 
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # 把Encoder最后时刻的隐状态作为Decoder的初始值
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # 因为我们的函数都是要求(time,batch)，因此即使只有一个数据，也要做出二维的。
        # Decoder的初始输入是SOS
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # 用于保存解码结果的tensor
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # 循环，这里只使用长度限制，后面处理的时候把EOS去掉了。
        for _ in range(max_length):
            # Decoder forward一步
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, 
								encoder_outputs)
            # decoder_outputs是(batch=1, vob_size)
            # 使用max返回概率最大的词和得分
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # 把解码结果保存到all_tokens和all_scores里
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # decoder_input是当前时刻输出的词的ID，这是个一维的向量，因为max会减少一维。
            # 但是decoder要求有一个batch维度，因此用unsqueeze增加batch维度。
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # 返回所有的词和得分。
        # print(all_tokens, all_scores)
        return all_tokens, all_scores

In [55]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### 把输入的一个batch句子变成id
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # 创建lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 转置 
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 放到合适的设备上(比如GPU)
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # 用searcher解码
    tokens, scores = searcher(input_batch, lengths, max_length)
    # ID变成词。
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    print(decoded_words)
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 得到用户终端的输入
            input_sentence = input('> ')
            # 是否退出
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 句子归一化
            input_sentence = normalizeString(input_sentence)
            # 生成响应Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 去掉EOS后面的内容
            words = []
            for word in output_words:
                if word == 'EOS':
                    break
                elif word != 'PAD':
                    words.append(word)
            print('Bot:', ''.join(words))

        except KeyError:
            print("Error: Encountered unknown word.")
      


In [60]:
# 配置模型
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# 从哪个checkpoint恢复，如果是None，那么从头开始训练。
loadFilename = '/content/drive/MyDrive/save/cb_model/qa_corpus.json/2-2_500/100_checkpoint.tar'
#loadFilename =None
checkpoint_iter = 100
  

# 如果loadFilename不空，则从中加载模型 
if loadFilename:
    # 如果训练和加载是一条机器，那么直接加载 
    #checkpoint = torch.load(loadFilename)
    # 否则比如checkpoint是在GPU上得到的，但是我们现在又用CPU来训练或者测试，那么注释掉下面的代码
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# 初始化word embedding
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# 初始化encoder和decoder模型
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, 
				decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# 使用合适的设备
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
# 配置训练的超参数和优化器 
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 100
corpus_name = "qa_corpus.json"
save_dir = os.path.join("/content/drive/MyDrive", "save")
# 设置进入训练模式，从而开启dropout 
encoder.train()
decoder.train()

# 初始化优化器 
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# 开始训练
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1187.)
  import sys
/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:149: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1401.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


Iteration: 101; Percent complete: 2.5%; Average loss: 6.6823
Iteration: 102; Percent complete: 2.5%; Average loss: 6.4915
Iteration: 103; Percent complete: 2.6%; Average loss: 6.5069
Iteration: 104; Percent complete: 2.6%; Average loss: 6.0861
Iteration: 105; Percent complete: 2.6%; Average loss: 6.1521
Iteration: 106; Percent complete: 2.6%; Average loss: 6.4271
Iteration: 107; Percent complete: 2.7%; Average loss: 6.6467
Iteration: 108; Percent complete: 2.7%; Average loss: 6.3677
Iteration: 109; Percent complete: 2.7%; Average loss: 6.4544
Iteration: 110; Percent complete: 2.8%; Average loss: 6.1741
Iteration: 111; Percent complete: 2.8%; Average loss: 6.7270
Iteration: 112; Percent complete: 2.8%; Average loss: 6.5960
Iteration: 113; Percent complete: 2.8%; Average loss: 6.3824
Iteration: 114; Percent complete: 2.9%; Average loss: 6.4397
Iteration: 115; Percent complete: 2.9%; Average loss: 6.3168
Iteration: 116; Percent complete: 2.9%; Average loss: 6.4083
Iteration: 117; Percent 

In [58]:
# 进入eval模式，从而去掉dropout。 
encoder.eval()
decoder.eval()

# 构造searcher对象 
searcher = GreedySearchDecoder(encoder, decoder) 

# 测试
evaluateInput(encoder, decoder, searcher, voc)

> 寻了个啥
['EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS', 'EOS']
Bot: 


KeyboardInterrupt: ignored